In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.metrics import precision_score, f1_score, recall_score


data =pd.read_csv("C:/Users/raimn/mlproject/res/ResNew1.csv")
print(data.head())
print(data.sample(5))

                                                text  polarity
0  اللي يوصل ميلان يمر هالمطعم اعتبره خسران خسران...         1
1  راءع المطعم اكله لذيذ جدا والعاملين متعاونين و...         1
2  جيد مطعم جيد العاملين ودون بالنسبه للاطعمه ممت...         1
3  مطعم جميل جدا ماكولات بجد جميله جدا فواتح شهيه...         1
4  مطعم مصري السلام عليكم المطعم ممتاز جدا مسلم ع...         1
                                                   text  polarity
3419  صراحه منب محبين البيتزا ابدا دومينوز وبيتزا هت...         1
6323  شخصيا اعتبره افضل مطاعم البرجر بريده وخصوصا بر...         1
3251                          مو الزود احلي والاكل عادي         1
1601  مطعم ناندوز مطعم راءع ولذيذ جذا يتميز بالدجاج ...         1
7340  اتذكر اول مره اشوفه علي طريق الملك عبدالعزيز ح...         1


In [71]:
from sklearnex import patch_sklearn 

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### splitting the data into target (text) and feature (polarity)

In [72]:
feature = data.text
target = data.polarity
# splitting into train and tests
X_train, X_test, Y_train, Y_test = train_test_split(feature.values.astype('U'), target.values.astype('U'), test_size =.2, random_state=100)


### TFIDF Vectorization and GridSearch
## Logistc Regression Model

In [62]:
# make pipeline, Sandard Features : TFIDF
pipe = make_pipeline(TfidfVectorizer(),
                    LogisticRegression())
# make param grid
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

# create and fit the model
model = GridSearchCV(pipe, param_grid, cv=5)
model.fit(X_train,Y_train,)

# make prediction and print accuracy
prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
#print(classification_report(Y_test, prediction))
print(classification_report(Y_test, prediction))



C:\Users\raimn\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\raimn\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Accuracy score is 0.86
              precision    recall  f1-score   support

          -1       0.80      0.67      0.73       541
           0       0.60      0.05      0.10        58
           1       0.87      0.95      0.91      1576

    accuracy                           0.86      2175
   macro avg       0.76      0.56      0.58      2175
weighted avg       0.85      0.86      0.84      2175



## Random Forest Model

In [63]:
pipe = make_pipeline(TfidfVectorizer(),
                    RandomForestClassifier())

param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
             'randomforestclassifier__max_features':['sqrt', 'log2']}

rf_model = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, verbose=1)
rf_model.fit(X_train,Y_train)

prediction = rf_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))


Fitting 5 folds for each of 6 candidates, totalling 30 fits
Accuracy score is 0.83
              precision    recall  f1-score   support

          -1       0.87      0.48      0.62       541
           0       0.00      0.00      0.00        58
           1       0.82      0.98      0.89      1576

    accuracy                           0.83      2175
   macro avg       0.57      0.49      0.50      2175
weighted avg       0.81      0.83      0.80      2175



C:\Users\raimn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raimn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raimn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Naive Bayes Model

In [82]:
#Also, naive Bayes has almost no hyperparameters to tune
pipe = make_pipeline(TfidfVectorizer(),
                    MultinomialNB())
pipe.fit(X_train,Y_train)
prediction = pipe.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.73
              precision    recall  f1-score   support

          -1       1.00      0.03      0.05       541
           0       0.00      0.00      0.00        58
           1       0.73      1.00      0.84      1576

    accuracy                           0.73      2175
   macro avg       0.58      0.34      0.30      2175
weighted avg       0.78      0.73      0.62      2175



C:\Users\raimn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raimn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raimn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Support Vector Model

In [73]:
pipe = make_pipeline(TfidfVectorizer(),
                     SVC())
param_grid = {'svc__kernel': ['rbf', 'linear', 'poly'],
             'svc__gamma': [0.1, 1, 10, 100],
             'svc__C': [0.1, 1, 10, 100]}

svc_model = GridSearchCV(pipe, param_grid, cv=3)
svc_model.fit(X_train, Y_train)

prediction = svc_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.86
              precision    recall  f1-score   support

          -1       0.81      0.70      0.75       541
           0       0.50      0.05      0.09        58
           1       0.88      0.95      0.91      1576

    accuracy                           0.86      2175
   macro avg       0.73      0.57      0.59      2175
weighted avg       0.85      0.86      0.85      2175

